## Drug Label - openFDA

JSON Download link : https://open.fda.gov/drug/label/

Extracted the following fields from the JSON files: [product_ndc, brand_name, manufacturer_name, product_type]

#### JQ - Bash
jq '.results[].openfda | "\(.product_ndc) @@ \(.generic_name) @@ \(.product_type) @@ \(.brand_name) @@ \(.manufacturer_name)"' 6.json > text6.txt

@@ used to seperate the fields. All 6 JSON files processed 

In [3]:
import pandas as pd
import glob, os
import numpy as np
import re

## Processing the text files

Read in the files and split on @@ - save this to a dataframe 

In [ ]:
dataframes = []
for file in glob.glob("*.txt"):
    df = pd.read_table(file,header=None)
    df = df[0].str.split('@@', -1, expand=True)
    dataframes.append(df)

Dropping the white spaces and replacing all the special charecters

In [ ]:
full = pd.concat(dataframes)
full.reset_index(drop=True)
full = full.apply(lambda x: x.str.strip().replace('null', np.nan))
cols = [i for i in full.columns if i not in ['product_ndc']]
for i in cols:
    full[i] = full[i].apply(lambda x: ''.join(str(x).replace('"', '').replace('\\', '').replace('[', '').replace(']', '')))
full.dropna(inplace=True)

In [ ]:
#Pickle the file.
#full.to_pickle('wip_drug_dataframe.pkl')

In [6]:
Drugs_label=pd.read_pickle('wip_drug_dataframe.pkl')
Drugs_label.head()

,product_ndc,generic_name,product_type,brand_name,manufacturer_name
0,"[\66336-706\""]",OMEPRAZOLE,HUMAN PRESCRIPTION DRUG,Omeprazole,"Dispensing Solutions, Inc."
1,"[\55045-1433\""]",CARISOPRODOL,HUMAN PRESCRIPTION DRUG,Carisoprodol,"Dispensing Solutions, Inc."
2,"[\59220-4001\""]",ADENOSINE,HUMAN OTC DRUG,NMC Staypot double programing cream,NMC (Natural Magma Cosmetics)
3,"[\0615-0547\""]",DIGOXIN,HUMAN PRESCRIPTION DRUG,Digoxin,"NCS HealthCare of KY, Inc dba Vangard Labs"
4,"[\66336-815\""]",NAPROXEN,HUMAN PRESCRIPTION DRUG,NAPROXEN,"Dispensing Solutions, Inc."


Drugs_label has product_ndc which has to be formatted and it is a list of lists.

In [7]:
#Extracting the list of product_ndc
Drugs_label.product_ndc=Drugs_label.product_ndc.apply(lambda x: re.findall('(\d+-\d+)',x))

## Merging the two datasets

The product_ndc column will be matched with the Labeler_Code and Product_code from the State Prescription dataset.
Two new fields will be created - NDC1 and NDC2.<br>
NDC1 will be "labeler_code" + "-" + "product_code"<br>
NDC2 will be "labeler_code" + "-" + "product_code's last 3 digits"<br>
This is done to later compare with the product_ndc column of Drugs_label dataset. The naming convention varies, so we are using this step to get the most number of matches.

In [9]:
state_pres = pd.read_pickle('unsuppressed_cleaned_state.pkl')
state_pres.head()

,Utilization_Type,State,Labeler_Code,Product_Code,Package_Size,Year,Quarter,Product_Name,Units_Reimbursed,Number_of_Prescriptions,Total_Amount_Reimbursed,Medicaid_Amount_Reimbursed,Non_Medicaid_Amount_Reimbursed
4,FFSU,AK,0002,3227,30,2014,1,STRATTERA,855.0,25.0,6880.17,5693.59,1186.58
5,FFSU,AK,0002,3227,30,2014,2,STRATTERA,762.0,21.0,6088.17,4760.39,1327.78
6,FFSU,AK,0002,3227,30,2014,3,STRATTERA,930.0,27.0,7327.43,6308.99,1018.44
7,FFSU,AK,0002,3227,30,2014,4,STRATTERA,942.0,23.0,7469.27,6751.02,718.25
8,FFSU,AK,0002,3228,30,2014,1,STRATTERA,3380.0,75.0,27707.80,25760.72,1947.08


In [10]:
#Making the NDC1 and NDC2 column
state_pres['NDC1']=state_pres.Labeler_Code+'-'+state_pres.Product_Code
state_pres['NDC2']=state_pres.Labeler_Code+'-'+state_pres.Product_Code.str[1:]

Function to flatten the list of product_ndc.

In [13]:
# We are using the brand_name and then merging with brand_name
def flatten(row):
    ndc = row['product_ndc'] if isinstance(row['product_ndc'], list) else [row['product_ndc']]
    s = pd.Series(row['brand_name'], index=list(set(ndc)))
    return s
# Running the function in batch to avoid memory error
#Splitting the dataframe into 10 parts
k=np.array_split(Drugs_label,10)
dfinal = pd.DataFrame()
for i in range(0,10):
    ex = k[i][['product_ndc', 'brand_name']]
    df1= ex.apply(flatten, axis=1).stack()
    df1 = df1.to_frame().reset_index(level=1, drop=False)
    df1.columns = ['product_ndc', 'brand_name']
    df1.reset_index(drop=True, inplace=True)
    dfinal = pd.concat([dfinal,df1])

In [17]:
#Merging the rest of the Drug label dataframe with brand name as the key
ex2 = Drugs_label[['generic_name', 'product_type', 'brand_name','manufacturer_name']]
output = dfinal.merge(ex2, on='brand_name', how='inner')
#Dropping the duplicates 
Drugs_label_final= output.drop_duplicates(['product_ndc', 'brand_name'])
#Saving this data frame as a pickle
#Drugs_label_final.to_pickle('Drugs_label_final.pkl')

<br>Merging the dataframe state_press with drug_label_final on both NDC1 and NDC2 and then concat the result<br>

In [19]:
x1 = Drugs_label_final.merge(state_pres, left_on='product_ndc', right_on='NDC1')
x2 = Drugs_label_final.merge(state_pres, left_on='product_ndc', right_on='NDC2')
state_pres_final = pd.concat([x1,x2])

In [21]:
state_pres_final.head()

,product_ndc,brand_name,generic_name,product_type,manufacturer_name,Utilization_Type,State,Labeler_Code,Product_Code,Package_Size,Year,Quarter,Product_Name,Units_Reimbursed,Number_of_Prescriptions,Total_Amount_Reimbursed,Medicaid_Amount_Reimbursed,Non_Medicaid_Amount_Reimbursed,NDC1,NDC2
0,0781-2234,Omeprazole,OMEPRAZOLE,HUMAN PRESCRIPTION DRUG,"Dispensing Solutions, Inc.",FFSU,AK,0781,2234,10,2014,1,OMEPRAZOLE,480.0,16.0,301.02,301.02,0.0,0781-2234,0781-234
1,0781-2234,Omeprazole,OMEPRAZOLE,HUMAN PRESCRIPTION DRUG,"Dispensing Solutions, Inc.",FFSU,AK,0781,2234,10,2014,2,OMEPRAZOLE,600.0,20.0,314.36,314.36,0.0,0781-2234,0781-234
2,0781-2234,Omeprazole,OMEPRAZOLE,HUMAN PRESCRIPTION DRUG,"Dispensing Solutions, Inc.",FFSU,AK,0781,2234,10,2014,3,OMEPRAZOLE,570.0,17.0,315.86,315.86,0.0,0781-2234,0781-234
3,0781-2234,Omeprazole,OMEPRAZOLE,HUMAN PRESCRIPTION DRUG,"Dispensing Solutions, Inc.",FFSU,AK,0781,2234,10,2014,4,OMEPRAZOLE,347.0,18.0,244.39,244.39,0.0,0781-2234,0781-234
4,0781-2234,Omeprazole,OMEPRAZOLE,HUMAN PRESCRIPTION DRUG,"Dispensing Solutions, Inc.",FFSU,AL,0781,2234,1,2014,1,OMEPRAZOLE,540.0,18.0,339.84,333.34,6.5,0781-2234,0781-234
